In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from astropy.table import Table, Column, vstack
import astropy.io.fits as pyfits
from astropy.io import ascii
import os
import re
from scipy import integrate, interpolate, optimize
import csv
matplotlib.rcParams['mathtext.fontset'] = 'cm' # computer modern
matplotlib.rcParams['font.family'] = 'STIXGeneral'

In [2]:
pathw = os.getcwd() + '/'
file_name = 'tlock.forward'

In [3]:
class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

        
        
# List of all subdirectories. 
# Change the parameter [0] depending on the needs (i.e., on the depth of the nested grid of subdirectories)
# It should always be x[1], which corresponds to the directories present in the current work path. 
# x[2] comprises the files present in the current work path


# planetary system
sdirect = [x[1] for x in os.walk(pathw)][0]
sdirect.remove('.ipynb_checkpoints')

# CPL/CTL
ssdirect = [[x[1] for x in os.walk(y)][0] for y in sdirect]

In [4]:
dim = (len(sdirect) , np.shape(ssdirect)[1]) # 22*2 = 44

direct = np.zeros(dim, dtype=object)

for i, sd in enumerate(sdirect):
    
    for j in range(np.shape(ssdirect)[1]):
        
                direct[i, j] = sd + '/' + ssdirect[i][j] + '/'
    

tot_models = np.prod(np.array(dim))
direct = np.reshape(direct, tot_models)

In [5]:
tlock = np.zeros(tot_models)
dSemi = np.zeros(tot_models)

sTideModel = np.zeros(tot_models, dtype=object)
dPrimaryMass = np.zeros(tot_models)
dPrimaryRadius = np.zeros(tot_models)
dSecondaryMass = np.zeros(tot_models)
dSecondaryRadius = np.zeros(tot_models)


# Python is stupid and I need to properly sort the directories
# https://ubuntuforums.org/showthread.php?t=2114348
for d, dd in enumerate(sorted(direct, key=lambda s: int(s.split("/")[0]))):
    
    tlock_forward = np.loadtxt(dd + '/tlock.forward')
    
    # Exclude 13 Gyr, which was the end of the run
    #if tlock_forward[-1][0] != 1.3E10:
    
    try:
        # In case any system went beyond 1.3E10 years
        if (tlock_forward[-1][0] <= 1.3E10):
    
            tlock[d] = tlock_forward[-1][0]
            dSemi[d] = tlock_forward[-1][1]

            # Do not read sOutputOrder Time Semim Ecce SecObli
            tidein = np.genfromtxt(dd + '/tide.in', dtype=str, skip_footer = 1)

            sTideModel[d] = tidein[1][1]
            dPrimaryMass[d] = float(tidein[-19][1])
            dPrimaryRadius[d] = -float(tidein[-18][1])
            dSecondaryMass[d] = -float(tidein[-11][1])
            dSecondaryRadius[d] = -float(tidein[-6][1])
        
        
        else:
        
            print dd + 'tlock beyond t_H'
            
    
    except IndexError: # System 14 is stopping at t=0
        
        print dd + 'tlock = 0'
        tlock[d] = 1.0e0 #tlock_forward[0]
        dSemi[d] = tlock_forward[1]

        # Do not read sOutputOrder Time Semim Ecce SecObli
        tidein = np.genfromtxt(dd + '/tide.in', dtype=str, skip_footer = 1)

        sTideModel[d] = tidein[1][1]
        dPrimaryMass[d] = float(tidein[-19][1])
        dPrimaryRadius[d] = -float(tidein[-18][1])
        dSecondaryMass[d] = -float(tidein[-11][1])
        dSecondaryRadius[d] = -float(tidein[-6][1])
        
        


        # e.g.
        #['sSystemName' 'proxima']
        #['sTideModel' 'CTL']
        #['bDiscreteRot' '1']
        #['iVerbose' '2']
        #['iDigits' '8']
        #['iSciNot' '4']
        #['bHaltSecLock' '1']
        #['bHaltSecSync' '0']
        #['bDoLog' '0']
        #['sUnitMass' 'solar']
        #['sUnitLength' 'AU']
        #['sUnitTime' 'year']
        #['sUnitAngle' 'degrees']
        #['bDoForward' '1']
        #['bVarDt' '1']
        #['dForwardStopTime' '13e9']
        #['dForwardOutputTime' '1e6']
        #['sForwardFile' 'tlock.forward']
        #['dTimestepCoeff' '0.01']
        #['dMinValue' '1e-10']
        #['dPrimaryMass' '0.7740']
        #['dPrimaryRadius' '-0.7149']
        #['dPrimarySpinPeriod' '-83']
        #['dPrimaryObliquity' '0']
        #['dPrimaryRadGyra' '0.5']
        #['dPrimaryK2' '0.5']
        #['dPrimaryQ' '1e6']
        #['dPrimaryTau' '-0.01']
        #['dSecondaryMass' '-37.0400']
        #['dSecondaryObliquity' '0']
        #['dSecondaryK2' '0.3']
        #['dSecondaryRadGyra' '0.5']
        #['dSecondarySpinPeriod' '-1']
        #['dSecondaryRadius' '-5.4900']
        #['dSecondaryQ' '12']
        #['dSecondaryTau' '-648']
        #['dSecondaryMaxLockDiff' '0.01']
        #['dSemi' '5.00']
        #['dEcc' '0.']

In [6]:
tlock = tlock[tlock != 0]
dSemi = dSemi[dSemi != 0]

sTideModel = sTideModel[sTideModel != 0]
dPrimaryMass = dPrimaryMass[dPrimaryMass != 0]
dPrimaryRadius = dPrimaryRadius[dPrimaryRadius != 0]
dSecondaryMass = dSecondaryMass[dSecondaryMass != 0]
dSecondaryRadius = dSecondaryRadius[dSecondaryRadius != 0]

In [7]:
dPrimaryLuminosity = np.zeros(tot_models)

for m, mm in enumerate(dPrimaryMass):
    
    if mm == np.max(dPrimaryMass):
        
        dPrimaryLuminosity[m] = 0.17256226
        
    elif mm == np.min(dPrimaryMass):
    
        dPrimaryLuminosity[m] = 0.0005378175
        
        
dPrimaryLuminosity = dPrimaryLuminosity[dPrimaryLuminosity != 0]

In [8]:
wh_CPL = np.where(sTideModel == 'CPL')[0]
wh_CTL = np.where(sTideModel == 'CTL')[0]

In [9]:
planet_ID = [4, 16, 17, 18, 21, 27, 29, 40, 43, 49, 52, 72, 76, 77, 81, 84, 93, 103, 106, 113, 115, 116, 124, 125, 148, 179, 187, 195, 196, 201, 202, 203, 204]

planet_name = ['LHS 1140', 'GJ 96', 'CD-23 1056', 'LP 413-32 B', 'LPM 178', 'LP 834-042', 'LP 834-042', 'LP 776-27', 'Kapteyns star', 'BD-06 1339', 'Luytens star', 'Innes star', 'PM J11293-0127', 'PM J11302+0735', 'Ross 1003', 'LP 613-39', 'Proxima Centauri', 'HD 147379', 'V2306 Oph', 'HD 156384 C', 'HD 156384 C', 'HD 156384 C', 'CD-44 11909', 'CD-44 11909', 'V1428 Aql', 'Kepler-186', 'HD 204961', 'IL Aqr', 'IL Aqr', '2MUCD 12171', '2MUCD 12171', '2MUCD 12171', '2MUCD 12171']

planet_char = ['b', 'b', 'b', 'b', 'c', 'b', 'd', 'c', 'b', 'c', 'b', 'b', 'd', 'b', 'b', 'b', 'b', 'b', 'c', 'c', 'e', 'f', 'b', 'c', 'b', 'f', 'c', 'b', 'c', 'd', 'e', 'f', 'g']

mplanet_vals = [6.65, 19.66, 114.00, 4.01, 6.80, 23.54, 7.60, 6.40, 4.80, 53.00, 2.89, 9.90, 11.10, 8.43, 96.70, 5.95, 1.27, 24.70, 3.41, 3.80, 2.70, 2.70, 4.40, 8.70, 12.20, 1.61, 5.40, 760.90, 241.50, 0.38, 0.76, 1.16, 1.45]

rplanet_vals = [1.43, 4.68, 13.22, 1.70, 2.51, 5.36, 2.67, 2.40, 2.03, 8.49, 1.50, 3.15, 1.51, 2.38, 12.02, 2.25, 1.09, 5.22, 1.69, 1.81, 1.43, 1.42, 1.97, 2.85, 3.57, 1.17, 2.19, 13.26, 14.04, 0.77, 0.92, 1.04, 1.13]

k2planet_vals = [0.30, 1.50, 1.50, 0.40, 0.81, 1.50, 0.89, 0.75, 0.57, 1.50, 0.30, 1.13, 0.31, 0.74, 1.50, 0.68, 0.30, 1.50, 0.40, 0.46, 0.30, 0.30, 0.54, 0.98, 1.34, 0.30, 0.65, 1.50, 1.50, 0.30, 0.30, 0.30, 0.30]

Qplanet_vals = [1.00e+2, 2.29e+4, 1.00e+6, 1.83e+2, 1.21e+3, 4.19e+4, 1.63e+3, 9.75e+2, 4.33e+2, 3.21e+5, 1.00e+2, 3.64e+3, 1.03e+2, 9.36e+2, 1.00e+6, 7.13e+2, 1.00e+2, 3.72e+4, 1.78e+2, 2.49e+2, 1.00e+2, 1.00e+2, 3.75e+2, 2.24e+3, 6.68e+3, 1.00e+2, 6.26e+2, 1.00e+6, 1.00e+6, 1.00e+2, 1.00e+2, 1.00e+2, 1.00e+2]

lambdaplanet_vals = [0.370, 0.233, 0.254, 0.358, 0.311, 0.235, 0.301, 0.317, 0.339, 0.246, 0.370, 0.273, 0.369, 0.318, 0.254, 0.326, 0.370, 0.235, 0.359, 0.352, 0.370, 0.370, 0.342, 0.291, 0.248, 0.370, 0.329, 0.254, 0.254, 0.370, 0.370, 0.370, 0.370]

tauplanet_vals = [7.66e+1, 3.34e-1, 7.66e-3, 4.17e+1, 6.32e+0, 1.83e-1, 4.68e+0, 7.85e+0, 1.77e+1, 2.38e-2, 7.66e+1, 2.10e+0, 7.41e+1, 8.18e+0, 7.66e-3, 1.07e+1, 7.66e+1, 2.06e-1, 4.30e+1, 3.08e+1, 7.66e+1, 7.66e+1, 2.04e+1, 3.41e+0, 1.15e+0, 7.66e+1, 1.22e+1, 7.66e-3, 7.66e-3, 7.66e+1, 7.66e+1, 7.66e+1, 7.66e+1]

mstar_vals = [0.1824, 0.5763, 0.6174, 0.5035, 0.4054, 0.4507, 0.4507, 0.4092, 0.2787, 0.6348, 0.2946, 0.3595, 0.5506, 0.4446, 0.3456, 0.3036, 0.1238, 0.6314, 0.3022, 0.3268, 0.3268, 0.3268, 0.2743, 0.2743, 0.4745, 0.5493, 0.4404, 0.3381, 0.3381, 0.0898, 0.0898, 0.0898, 0.0898]

rstar_vals = [0.2306, 0.5535, 0.6275, 0.6208, 0.4245, 0.4517, 0.4517, 0.4237, 0.2754, 0.6390, 0.3237, 0.3856, 0.5381, 0.4419, 0.3984, 0.3000, 0.1666, 0.6849, 0.3183, 0.3325, 0.3325, 0.3325, 0.2932, 0.2932, 0.4909, 0.5236, 0.4393, 0.3699, 0.3699, 0.1236, 0.1236, 0.1236, 0.1236]

a_vals = [0.087, 0.291, 0.250, 0.164, 0.125, 0.143, 0.194, 0.129, 0.168, 0.457, 0.091, 0.119, 0.208, 0.143, 0.166, 0.091, 0.049, 0.319, 0.089, 0.125, 0.213, 0.156, 0.080, 0.176, 0.336, 0.432, 0.163, 0.214, 0.134, 0.021, 0.028, 0.037, 0.045]

star_L = [0.004425108, 0.063856810, 0.090805605, 0.036404450, 0.021697850, 0.027579645, 0.027579645, 0.024274724, 0.012805443, 0.103932050, 0.009900130, 0.017896352, 0.060347500, 0.026395837, 0.014989760, 0.012170071, 0.001536843, 0.097754300, 0.010823072, 0.014945071, 0.014945071, 0.014945071, 0.008118322, 0.008118322, 0.032581307, 0.057127886, 0.029206133, 0.012926352, 0.012926352, 0.000537818, 0.000537818, 0.000537818, 0.000537818]

star_Teff = [3100, 3900, 4000, 3200, 3400, 3500, 3500, 3500, 3700, 4100, 3200, 3400, 3900, 3500, 3200, 3500, 2800, 3900, 3300, 3500, 3500, 3500, 3200, 3200, 3500, 3900, 3600, 3200, 3200, 2500, 2500, 2500, 2500]

In [10]:
# ID,Name,Planet,a_au,M_star,Lbol,Teff,M_planet,R_planet, tlock_CPL, tlock_CTL

with open('hmr18.planets_tlock.v08.csv', 'w') as f:
    
    row = csv.writer(f)
    
    for p in range(len(tlock[wh_CPL])):
        row.writerow([planet_ID[p], planet_name[p], planet_char[p],\
                     a_vals[p], mstar_vals[p], star_L[p],\
                     star_Teff[p], mplanet_vals[p], rplanet_vals[p],\
                     "%.4e" % tlock[wh_CPL][p], "%.4e" % tlock[wh_CTL][p]])